In [1]:
from datetime import datetime
import os

from datasets import load_dataset
from dotenv import load_dotenv

import pandas as pd
import requests


load_dotenv()

True

In [2]:
dataset = load_dataset("DataForGood/climateguard")

README.md:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/542k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/568 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/173 [00:00<?, ? examples/s]

In [3]:
df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df = pd.concat([df_train, df_test])


In [4]:
display(df_train.channel.value_counts(normalize=True).round(3))
display(df_test.channel.value_counts(normalize=True).round(3))

channel
itele             0.285
sud-radio         0.132
europe1           0.114
franceinfotv      0.070
lci               0.067
rmc               0.063
france-info       0.062
france24          0.051
bfmtv             0.033
france-inter      0.026
tf1               0.021
france-culture    0.019
rtl               0.016
rfi               0.014
france2           0.012
arte              0.009
fr3-idf           0.002
m6                0.002
Name: proportion, dtype: float64

channel
itele             0.312
sud-radio         0.162
europe1           0.116
franceinfotv      0.069
rmc               0.064
bfmtv             0.052
france-info       0.052
lci               0.040
france24          0.040
france2           0.035
tf1               0.023
arte              0.017
rtl               0.006
france-culture    0.006
france-inter      0.006
Name: proportion, dtype: float64

In [5]:
train_counts = df_train.groupby("channel").agg(count=("id", "count")).reset_index()
test_counts = df_test.groupby("channel").agg(count=("id", "count")).reset_index()
counts = pd.merge(
    left=train_counts,
    right=test_counts,
    on="channel",
    how="outer",
    suffixes=["_train", "_test"],
)

In [6]:
counts

,channel,count_train,count_test
0,arte,5,3.0
1,bfmtv,19,9.0
2,europe1,65,20.0
3,fr3-idf,1,NaN
4,france-culture,11,1.0
5,france-info,35,9.0
6,france-inter,15,1.0
7,france2,7,6.0
8,france24,29,7.0
9,franceinfotv,40,12.0


In [7]:
import plotly.express as px

display(
    px.bar(
        counts, x="channel", y=["count_train", "count_test"], barmode="group", log_y=True
    ).update_xaxes(
        categoryorder="array", categoryarray=sorted(df_train.channel.unique())
    ).update_yaxes(title="Count")
)

In [8]:
train_perc = (
    df_train.groupby("channel").agg(percentage=("id", "count")) / len(df_train) * 100
).round(2).reset_index()
test_perc = (
    df_test.groupby("channel").agg(percentage=("id", "count")) / len(df_test) * 100
).round(2).reset_index()
perc = pd.merge(
    left=train_perc,
    right=test_perc,
    on="channel",
    how="outer",
    suffixes=["_train", "_test"],
)

In [9]:
display(
    px.bar(
        perc, x="channel", y=["percentage_train", "percentage_test"], barmode="group", log_y=True
    ).update_xaxes(
        categoryorder="array", categoryarray=sorted(df_train.channel.unique())
    ).update_yaxes(title="% of Examples (Log)")
)

In [10]:
train_perc = (
    df_train.groupby("misinformation").agg(percentage=("id", "count"))
).round(2).reset_index()
test_perc = (
    df_test.groupby("misinformation").agg(percentage=("id", "count"))
).round(2).reset_index()
perc = pd.merge(
    left=train_perc,
    right=test_perc,
    on="misinformation",
    how="outer",
    suffixes=["_train", "_test"],
)
display(
    px.bar(
        perc, x="misinformation", y=["percentage_train", "percentage_test"], barmode="group"
    ).update_xaxes(
        categoryorder="array", categoryarray=sorted(df_train.channel.unique())
    ).update_yaxes(title="Count")
)

In [11]:
train_perc = (
    df_train.groupby("misinformation").agg(percentage=("id", "count")) / len(df_train) * 100
).round(2).reset_index()
test_perc = (
    df_test.groupby("misinformation").agg(percentage=("id", "count")) / len(df_test) * 100
).round(2).reset_index()
perc = pd.merge(
    left=train_perc,
    right=test_perc,
    on="misinformation",
    how="outer",
    suffixes=["_train", "_test"],
)
display(
    px.bar(
        perc, x="misinformation", y=["percentage_train", "percentage_test"], barmode="group"
    ).update_xaxes(
        categoryorder="array", categoryarray=sorted(df_train.channel.unique())
    ).update_yaxes(title="Percentage of Examples (%)")
)